In [ ]:
#Install gensim dependency
!pip install gensim

In [24]:
#Import all the dependencies
import gensim
import nltk
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from os import listdir
from os.path import isfile, join
from nltk.stem.wordnet import WordNetLemmatizer
import string

In [ ]:
#Download stop words and wordnet dependencies
nltk.download()

In [72]:
#now create a list that contains the labels of all the text files based on their folders
# and another list that has the document contents
docLabels = []
#docLabels = [f for f in listdir("datas/")]
#print(docLabels)
#create a list data that stores the content of all text files in order of their names in docLabels
data = []
for category in listdir("datas/"):
    #counter = 0
    for doc in listdir("datas/" + category):
        #if(counter < 20):
        docLabels.append(category)
        counter += 1
        with open("datas/" + category + "/" + doc,encoding='utf-8', errors='ignore') as f:
            data.append(f.read())
            #txt = f.read()


In [80]:
#Cleaning function fro the data
tokenizer = RegexpTokenizer(r'\w+')
stopword_set = set(stopwords.words('english'))
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
#This function does all cleaning of data using two objects above
def nlp_clean(data):
   new_data = []
   for d in data:
        stop_free = " ".join([i for i in d.lower().split() if i not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
        new_str = normalized.lower()
        dlist = tokenizer.tokenize(new_str)
        dlist = list(set(dlist).difference(stopword_set))
        new_data.append(normalized)
   return new_data

In [54]:
#TaggedDocument class formats the document contents into a tagged format where documents 
# are tagged based on the folder they are in
class TaggedDocument(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
              yield gensim.models.doc2vec.TaggedDocument(doc,    
[self.labels_list[idx]])

In [74]:
#Clean the data
data = nlp_clean(data)

In [75]:
#iterator returned over all documents
it = TaggedDocument(data, docLabels)

In [79]:
#Find word count
word_count = 0
for i in data:
    word_count += len(i)
print(word_count)

29475


In [78]:
#Set up model and trian it
model = gensim.models.Doc2Vec(vector_size=300, min_count=0, alpha=0.025, min_alpha=0.025)
model.build_vocab(it)
#training of model
for epoch in range(25):
 print('iteration ' + str(epoch+1))
 model.train(it, total_words=29475, epochs=1)
 model.alpha -= 0.002
 model.min_alpha = model.alpha
#saving the created model
model.save('doc2vec.model')
print("model saved")

iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
model saved


In [116]:
#Gather test data
test_data =[]
filenames = []
for doc in listdir("datas/Sexual violence"):
    filenames.append(doc)
    with open("datas/Sexual violence/" + doc,encoding='utf-8', errors='ignore') as f:
        test_data.append(f.read())

In [117]:
#Clean test data
test_data = nlp_clean(test_data)

In [114]:
#loading the model
#d2v_model = gensim.models.doc2vec.Doc2Vec.load('doc2vec2.model')

#start testing
#printing the vector of document at index 1 in docLabels
#docvec = d2v_model.docvecs[1]
#print(docvec)

#printing the vector of the file using its name
#docvec = d2v_model.docvecs['Physical violence'] #if string tag used in training
#print(docvec)

#to get most similar document with similarity scores using document-index
#similar_doc = d2v_model.docvecs.most_similar(10) 
#print(similar_doc)


#Check to find most similar tags for a new doc in the test_data
def check_single(test_data, index):
    new_vector = d2v_model.infer_vector(test_data[index].split());
    sims = d2v_model.docvecs.most_similar([new_vector])
    print("Filename: ", filenames[index])
    print(sims, "\n")

#sims = d2v_model.wv.most_similar(positive=['bullying', 'dating'], negative=['violence'])
#print(sims)
#to get vector of document that are not present in corpus 
#docvec = model.docvecs.infer_vector('war.txt')
#print(docvec)

In [118]:
#Find topiic label scores for each new document
for i in range(len(test_data)):
    check_single(test_data, i)

Filename:  Chapter 3 -- The Developmental Dynamics of Youth Violence - Youth Violence - NCBI Bookshelf.txt
[('Gang activities', -0.9269986152648926), ('Emotional violence', -0.9320201873779297), ('Verbal violence', -0.9327189326286316), ('Stealing', -0.9360224008560181), ('Physical violence', -0.9393171072006226), ('Bullying', -0.9393727779388428), ('Aggression', -0.9399834871292114), ('Sexual violence', -0.9414368867874146), ('Cultural violence', -0.9491147994995117)] 

Filename:  166363.txt
[('Stealing', -0.9165915846824646), ('Gang activities', -0.9196247458457947), ('Sexual violence', -0.925790011882782), ('Emotional violence', -0.9260824918746948), ('Verbal violence', -0.9287612438201904), ('Cultural violence', -0.930221676826477), ('Bullying', -0.9307388067245483), ('Aggression', -0.934402346611023), ('Physical violence', -0.9393256306648254)] 

Filename:  doc7.txt
[('Gang activities', -0.9679570198059082), ('Stealing', -0.9717816114425659), ('Physical violence', -0.9726495742797

Filename:  Interparental Violence and Maternal Modd Disorders....txt
[('Gang activities', -0.8730509877204895), ('Emotional violence', -0.8779202699661255), ('Verbal violence', -0.87940514087677), ('Physical violence', -0.881520688533783), ('Bullying', -0.8851324319839478), ('Aggression', -0.8856866359710693), ('Stealing', -0.8883217573165894), ('Sexual violence', -0.8894681930541992), ('Cultural violence', -0.9002925753593445)] 

Filename:  SH_2.txt
[('Gang activities', -0.9053711295127869), ('Verbal violence', -0.9109619855880737), ('Emotional violence', -0.9150579571723938), ('Physical violence', -0.9165488481521606), ('Bullying', -0.918935239315033), ('Aggression', -0.9206461906433105), ('Stealing', -0.9209768176078796), ('Sexual violence', -0.9271434545516968), ('Cultural violence', -0.9308751821517944)] 

Filename:  SH_7.txt
[('Gang activities', -0.909734845161438), ('Physical violence', -0.9105868339538574), ('Emotional violence', -0.9130938649177551), ('Verbal violence', -0.913

Filename:  SH_8.txt
[('Gang activities', -0.9113982915878296), ('Stealing', -0.9184138774871826), ('Verbal violence', -0.9184567332267761), ('Emotional violence', -0.9189378023147583), ('Physical violence', -0.9234791398048401), ('Bullying', -0.9238653779029846), ('Aggression', -0.9239094257354736), ('Cultural violence', -0.9243084192276001), ('Sexual violence', -0.9260176420211792)] 

Filename:  A Review of the Extent, Nature, Characteristics and Effects.txt
[('Gang activities', -0.9509466886520386), ('Verbal violence', -0.9530205726623535), ('Physical violence', -0.9537501931190491), ('Emotional violence', -0.954010009765625), ('Bullying', -0.9552175998687744), ('Aggression', -0.9571136236190796), ('Stealing', -0.9577778577804565), ('Sexual violence', -0.960680365562439), ('Cultural violence', -0.9621178507804871)] 

Filename:  WEAPONS USE GANG ACTIVITIES FIGHTING Onset and Prevalence of Serious Violence.txt
[('Gang activities', -0.9422422647476196), ('Verbal violence', -0.9508317708

Filename:  SH_10.txt
[('Verbal violence', -0.8623725175857544), ('Gang activities', -0.8625586032867432), ('Emotional violence', -0.8680789470672607), ('Physical violence', -0.8691211342811584), ('Bullying', -0.8749215602874756), ('Aggression', -0.8761502504348755), ('Stealing', -0.8826104998588562), ('Sexual violence', -0.8879496455192566), ('Cultural violence', -0.8996853232383728)] 

Filename:  verbal bullying.txt
[('Gang activities', -0.9707003831863403), ('Stealing', -0.971926212310791), ('Verbal violence', -0.9719481468200684), ('Cultural violence', -0.9728862047195435), ('Emotional violence', -0.9751116037368774), ('Physical violence', -0.9752290844917297), ('Sexual violence', -0.9760638475418091), ('Bullying', -0.9766052961349487), ('Aggression', -0.976940393447876)] 

Filename:  Cross-Study Violence.txt
[('Gang activities', -0.9476087093353271), ('Stealing', -0.9515244960784912), ('Verbal violence', -0.952666699886322), ('Cultural violence', -0.9540101289749146), ('Emotional v

Filename:  peer relations.txt
[('Gang activities', -0.9067741632461548), ('Verbal violence', -0.911829948425293), ('Physical violence', -0.912161648273468), ('Bullying', -0.9140596985816956), ('Emotional violence', -0.9141061305999756), ('Stealing', -0.9167857766151428), ('Aggression', -0.9170026779174805), ('Cultural violence', -0.9225912690162659), ('Sexual violence', -0.9228559732437134)] 

Filename:  WEAPONS USE Movies A way of promoting school violence.txt
[('Verbal violence', -0.8181561231613159), ('Gang activities', -0.8195130825042725), ('Physical violence', -0.8268042802810669), ('Emotional violence', -0.8283935785293579), ('Bullying', -0.8299511671066284), ('Aggression', -0.8336551189422607), ('Stealing', -0.8338141441345215), ('Sexual violence', -0.8435765504837036), ('Cultural violence', -0.8549426794052124)] 

Filename:  Chapter 3.6 -- The Developmental Dynamics of Youth Violence - Youth Violence - NCBI Bookshelf.txt
[('Gang activities', -0.926385760307312), ('Emotional vi